In [2]:
import json
import networkx as nx
import random
import plotly.graph_objects as go

RANDOM_SEED = 42
random.seed(RANDOM_SEED)

graph = nx.Graph()

f = json.load(open('run_1_gpt2_medium_graph_rl_input_graph.json'))
graph.add_nodes_from(f['nodes'])
graph.add_edges_from(f['edges'])

removed_nodes = [i for i in range(50, len(graph.nodes()))]
graph.remove_nodes_from(removed_nodes)

pos = nx.random_layout(graph, seed=RANDOM_SEED)
for n, p in pos.items():
    graph.nodes[n]['pos'] = p

node_x = []
node_y = []
for node in graph.nodes():
    x, y = graph.nodes[node]['pos']
    node_x.append(x)
    node_y.append(y)

edge_x = []
edge_y = []
for edge in graph.edges():
    x0, y0 = graph.nodes[edge[0]]['pos']
    x1, y1 = graph.nodes[edge[1]]['pos']
    edge_x.append(x0)
    edge_x.append(x1)
    edge_x.append(None)
    edge_y.append(y0)
    edge_y.append(y1)
    edge_y.append(None)

edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=0.5, color='#888'),
    hoverinfo='none',
    mode='lines')

node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers',
    hoverinfo='text',
    marker=dict(
        showscale=True,
        colorscale='Jet',
        reversescale=True,
        color=[],
        size=10,
        colorbar=dict(
            thickness=15,
            title='Eigenvector Centrality',
            xanchor='left',
            titleside='right'
        ),
        line_width=2))

centrality = nx.eigenvector_centrality(graph)
node_centralities = [centrality[node] for node in centrality]
node_text = [graph.nodes[node]["sentences"] for node in graph.nodes()]

node_trace.marker.color = node_centralities
node_trace.text = node_text

fig = go.Figure(data=[edge_trace, node_trace],
             layout=go.Layout(
                title='<br>sentence similarity seed subgraph (first 50) based on cosine similarities',
                titlefont_size=16,
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
                )

                
fig.show()
fig.write_html("graph_gpt2.html")

In [3]:
import json
import networkx as nx
import random
import plotly.graph_objects as go

RANDOM_SEED = 42
random.seed(RANDOM_SEED)

graph = nx.Graph()

f = json.load(open('run_1_gpt2_medium_graph_rl_output_graph.json'))
graph.add_nodes_from(f['nodes'])
graph.add_edges_from(f['edges'])

removed_nodes = [i for i in range(0, len(graph.nodes()) - 50)]
graph.remove_nodes_from(removed_nodes)

pos = nx.random_layout(graph, seed=RANDOM_SEED)
for n, p in pos.items():
    graph.nodes[n]['pos'] = p

node_x = []
node_y = []
for node in graph.nodes():
    x, y = graph.nodes[node]['pos']
    node_x.append(x)
    node_y.append(y)

edge_x = []
edge_y = []
for edge in graph.edges():
    x0, y0 = graph.nodes[edge[0]]['pos']
    x1, y1 = graph.nodes[edge[1]]['pos']
    edge_x.append(x0)
    edge_x.append(x1)
    edge_x.append(None)
    edge_y.append(y0)
    edge_y.append(y1)
    edge_y.append(None)

edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=0.5, color='#888'),
    hoverinfo='none',
    mode='lines')

node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers',
    hoverinfo='text',
    marker=dict(
        showscale=True,
        colorscale='Jet',
        reversescale=True,
        color=[],
        size=10,
        colorbar=dict(
            thickness=15,
            title='Eigenvector Centrality',
            xanchor='left',
            titleside='right'
        ),
        line_width=2))

centrality = nx.eigenvector_centrality(graph)
node_centralities = [centrality[node] for node in centrality]
node_text = [graph.nodes[node]["sentences"] for node in graph.nodes()]

node_trace.marker.color = node_centralities
node_trace.text = node_text

fig = go.Figure(data=[edge_trace, node_trace],
             layout=go.Layout(
                title='<br>sentence similarity updated subgraph (last generated 50)',
                titlefont_size=16,
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
                )

                
fig.show()

In [8]:
import json
import networkx as nx
import random
import numpy as np
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

RANDOM_SEED = 42
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

graph = nx.Graph()

f = json.load(open('1_med_output_graph.json'))
graph.add_nodes_from(f['nodes'])
graph.add_edges_from(f['edges'])

node_embeddings = [graph.nodes[n]["cls_embeddings"] for n in graph.nodes()]
for i in range(len(node_embeddings)):
    if isinstance(node_embeddings[i], float):
        graph.remove_node(i)
node_embeddings = [graph.nodes[n]["cls_embeddings"] for n in graph.nodes()]

node_colors = ["original" if graph.nodes[n]["plot_labels"] == 0 else "generated" for n in graph.nodes()]

cnt_original = 0
cnt_generated = 0
for i in node_colors:
    if i == "original":
        cnt_original += 1
    else:
        cnt_generated += 1

print("Number of original sentences in graph = " + str(cnt_original))
print("Number of generated sentences in graph = " + str(cnt_generated))

node_texts = [graph.nodes[n]["sentences"] for n in graph.nodes()]

tsne = TSNE(n_components=2, n_iter=5000)
node_embeddings_2d = tsne.fit_transform(node_embeddings)

tsne_df = pd.DataFrame({"x": node_embeddings_2d[:, 0], "y": node_embeddings_2d[:, 1], "labels": node_colors, "node_texts": node_texts})

fig = px.scatter(tsne_df, x="x", y="y", color="labels", hover_data=["node_texts"])
fig.show()
fig.write_html("gen_cls_tsne.html")

Number of original sentences in graph = 444
Number of generated sentences in graph = 173


In [2]:
import plotly.io as pio
pio.write_image(fig, 'gen_cls_tsne.pdf', width=700, height=775, engine="kaleido")